In [ ]:
%matplotlib inline

import os
import sys
import time
import six
import simulator as sim
import numpy as np
import datetime as dt

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import colors

SEC = 1000000
MIN = 60 * SEC
DOLLAR = 1000000
CENT = 10000

DATA_PATH = "data/"
OUTPUT_PATH = "output/"


################################################################################
# Visualization
################################################################################


# Colors list

colors_ = list(six.iteritems(colors.cnames))
# Add the single letter colors.
for name, rgb in six.iteritems(colors.ColorConverter.colors):
    hex_ = colors.rgb2hex(rgb)
    colors_.append((name, hex_))
# Transform to hex color values.
hex_ = [color[1] for color in colors_]
# Get the rgb equivalent.
rgb = [colors.hex2color(color) for color in hex_]
# Get the hsv equivalent.
hsv = [colors.rgb_to_hsv(color) for color in rgb]
# Split the hsv values to sort.
hue = [color[0] for color in hsv]
sat = [color[1] for color in hsv]
val = [color[2] for color in hsv]
# Sort by hue, saturation and value.
ind = np.lexsort((val, sat, hue))
sorted_colors = [colors_[i] for i in ind]

# Get dates

files = os.listdir(OUTPUT_PATH)
dates = []
for f in files:
    if f.endswith(".npz"):
        dates.append(f.split('_')[0])
dates.sort()
dates = dates[10:30]
dates_dt = [dt.datetime.strptime(x, "%Y%m%d").date() for x in dates]

# Get output data

ML_ALGO = "RF"
output = {}
sample_times = []
start = True
tickers = []
for date in dates:
    output_filename = "{0}{1}_{2}_output.npz".format(OUTPUT_PATH, date, ML_ALGO)
    output_file = np.load(output_filename)    
    if start:
        start = False
        tickers = output_file.files
        sample_times = output_file[tickers[0]][:, 0].astype(int).tolist()
        for ticker in tickers:
            output[ticker] = output_file[ticker]
    else:    
        for ticker in tickers:
            temp = output_file[ticker]
            output[ticker] = np.concatenate((output[ticker], temp), axis=0)

# Visualize PNL

last_indices = (output[tickers[0]][:, 0] == sample_times[-1])
pnl = {}
pnl_unr = {}
for ticker in tickers:
    output_last = output[ticker][last_indices, :]
    # PNL = cash + book value
    pnl[ticker] = [x / DOLLAR for x in (output_last[:, 1] +  output_last[:, 3]).astype(int).tolist()]
    # Unrealized PNL = cash + inventory * market value
    pnl_unr[ticker] = [x / DOLLAR for x in (output_last[:, 1] + output_last[:, 2] * output_last[:, 4]).astype(int).tolist()]
datefmt = mdates.DateFormatter("%m/%d")
dateloc = mdates.WeekdayLocator()

# PNL

fig, ax = plt.subplots()
fig.autofmt_xdate()
color_index = 0
y_min = -1
y_max = 1
for ticker in tickers:
    ax.plot(dates_dt, pnl[ticker], color=hex_[color_index], label=ticker)
    color_index = color_index + 10
    ticker_min = np.min(pnl[ticker]) - 1
    ticker_max = np.max(pnl[ticker]) + 1
    y_min = ticker_min if (ticker_min < y_min) else y_min
    y_max = ticker_max if (ticker_max > y_max) else y_max
ax.axhline(color="k")
ax.xaxis.set_major_formatter(datefmt)
ax.xaxis.set_major_locator(dateloc)
plt.ylim([y_min, y_max])
plt.legend(tickers)
ax.set(title="Daily PNL with {}".format(ML_ALGO), ylabel="PNL ($)", xlabel="Date")
# plt.savefig("{0}{1}_pnl.png".format(OUTPUT_PATH, ML_ALGO), dpi=300)
plt.show()

# Visualize inventory

sample_times_dt = [sim.micro_to_time(x) for x in sample_times]
sample_times_dt = [dt.datetime.combine(dt.date(2000, 1, 1), x) for x in sample_times_dt]
inventory = {}
for ticker in tickers:
    inventory[ticker] = []
    for time in sample_times:
        inventory[ticker].append(np.mean(output[ticker][output[ticker][:, 0] == time, 2]))
timefmt = mdates.DateFormatter("%H:%M")
timeloc = mdates.HourLocator()
        
fig, ax = plt.subplots()
fig.autofmt_xdate()
color_index = 0
y_min = -1
y_max = 1
for ticker in tickers:
    ax.plot(sample_times_dt, inventory[ticker], color=hex_[color_index], label=ticker)
    color_index = color_index + 10
    ticker_min = np.min(inventory[ticker]) - 1
    ticker_max = np.max(inventory[ticker]) + 1
    y_min = ticker_min if (ticker_min < y_min) else y_min
    y_max = ticker_max if (ticker_max > y_max) else y_max
ax.axhline(color="k")
ax.xaxis.set_major_formatter(timefmt)
ax.xaxis.set_major_locator(timeloc)
plt.ylim([y_min, y_max])
plt.legend(tickers, loc="upper left")
ax.set(title="Average inventory with {0}".format(ML_ALGO), ylabel="Stocks", xlabel="Time")
plt.savefig("{0}{1}_inventory.png".format(OUTPUT_PATH, ML_ALGO), dpi=300)
plt.show()

# Visualize cumulative PNL

cum_pnl = {}
cum_pnl_unr = {}
for ticker in tickers:
    # PNL = cash + book value
    cum_pnl[ticker] = np.cumsum(pnl[ticker])
    # Unrealized PNL = cash + inventory * market value
    cum_pnl_unr[ticker] = np.cumsum(pnl_unr[ticker])

fig, ax = plt.subplots()
fig.autofmt_xdate()
color_index = 0
y_min = -1
y_max = 1
for ticker in tickers:
    ax.plot(dates_dt, cum_pnl[ticker], color=hex_[color_index], label=ticker)
    color_index = color_index + 10
    ticker_min = np.min(cum_pnl[ticker]) - 1
    ticker_max = np.max(cum_pnl[ticker]) + 1
    y_min = ticker_min if (ticker_min < y_min) else y_min
    y_max = ticker_max if (ticker_max > y_max) else y_max
ax.axhline(color="k")
ax.xaxis.set_major_formatter(datefmt)
ax.xaxis.set_major_locator(dateloc)
plt.ylim([y_min, y_max])
plt.legend(tickers)
ax.set(title="Cumulative PNL with {0}".format(ML_ALGO), ylabel="PNL ($)", xlabel="Date")
plt.savefig("{0}{1}_cumpnl.png".format(OUTPUT_PATH, ML_ALGO), dpi=300)
plt.show()
